In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Attack_Prediction.csv')

In [2]:
columns=df.columns

for column in columns:
    if df[column].dtype=="int32":
        df[column]=df[column].astype("int16")
    elif df[column].dtype=="float64":
        df[column]=df[column].astype("float16")
    elif df[column].dtype=="object":
        df[column]=df[column].astype("category")

In [3]:
df['Sex']= df['Sex'].map({'Female': 0, 'Male': 1})
df['Sex']= pd.to_numeric(df['Sex'])

df['Diet']= df['Diet'].map({'Healthy': 0, 'Average': 1, 'Unhealthy':2})
df['Diet']= pd.to_numeric(df['Diet'])

df[['HBP', 'LBP']]= df['Blood Pressure'].str.split('/', expand= True)
df['HBP']= pd.to_numeric(df['HBP'])
df['LBP']= pd.to_numeric(df['LBP'])

df['Diabetes'] = df['Diabetes'].map({0: 1, 1: 0})

df['Exercise Hours Per Week']= round(df['Exercise Hours Per Week'], 0)

df['Sedentary Hours Per Day']= round(df['Sedentary Hours Per Day'], 0)

df['Income']= round(df['Income'], 0)

df['BMI']= round(df['BMI'], 0)

df = df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere'])

In [4]:
X= df.drop(['Heart Attack Risk'], axis= 1)
y= df['Heart Attack Risk']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

from sklearn.preprocessing import RobustScaler
scaler= RobustScaler()
scaler.fit(X_train)

X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)

In [6]:
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans

sm= ClusterCentroids(random_state= 15, estimator= KMeans(n_init= 10))
tl= SMOTE(random_state= 15)

X_sm, y_sm= sm.fit_resample(X_train, y_train)
X_sm_tl, y_sm_tl= tl.fit_resample(X_sm, y_sm)

In [7]:
from collections import Counter

original_distribution = Counter(y_train)
print(f"Original class distribution: {original_distribution}")


resampled_distribution = Counter(y_sm_tl)
print(f"Resampled class distribution: {resampled_distribution}")

Original class distribution: Counter({0: 4499, 1: 2511})
Resampled class distribution: Counter({0: 2511, 1: 2511})


--- LogisticRegression ---

In [8]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)

sfs= SequentialFeatureSelector(lr, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features= sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

lr.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lr, labels= lr.classes_)
print(classification_report(y_test, y_pred_lr, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr))

              precision    recall  f1-score   support

           0       0.64      0.41      0.50      1125
           1       0.36      0.59      0.44       628

    accuracy                           0.47      1753
   macro avg       0.50      0.50      0.47      1753
weighted avg       0.54      0.47      0.48      1753

Precision: 0.35658914728682173
Accuracy: 0.47290359383913294
AUC: 0.4983595187544232


--- DecisionTreeClassifier ---

In [9]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)

sfs= SequentialFeatureSelector(dt, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features= sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_dt, labels= dt.classes_)
print(classification_report(y_test, y_pred_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt))

              precision    recall  f1-score   support

           0       0.64      0.39      0.49      1125
           1       0.36      0.61      0.45       628

    accuracy                           0.47      1753
   macro avg       0.50      0.50      0.47      1753
weighted avg       0.54      0.47      0.47      1753

Precision: 0.35747883349012227
Accuracy: 0.46891043924700515
AUC: 0.4992668082094833


--- Tuned - DecisionTreeClassifier ---

In [10]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 20}
Best Precision Score: 0.5921628343584575


In [11]:
tuned_dt= gs_dt.best_estimator_

sfs= SequentialFeatureSelector(tuned_dt, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features = sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]
tuned_dt.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

              precision    recall  f1-score   support

           0       0.61      0.30      0.40      1125
           1       0.35      0.66      0.45       628

    accuracy                           0.43      1753
   macro avg       0.48      0.48      0.43      1753
weighted avg       0.52      0.43      0.42      1753

Precision: 0.34551495016611294
Accuracy: 0.4295493439817456
AUC: 0.4778549186128804


--- KNeighborsClassifier ---

In [12]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()

sfs= SequentialFeatureSelector(knn, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features= sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

knn.fit(X_sm_tl_selected, y_sm_tl)
y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_knn, labels= knn.classes_)
print(classification_report(y_test, y_pred_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn))

C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is i

C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is i

              precision    recall  f1-score   support

           0       0.62      0.36      0.45      1125
           1       0.35      0.61      0.44       628

    accuracy                           0.45      1753
   macro avg       0.48      0.48      0.45      1753
weighted avg       0.52      0.45      0.45      1753

Precision: 0.34660633484162895
Accuracy: 0.44837421563034796
AUC: 0.4908754423213022


--- Tuned - KNeighborsClassifier ---

In [13]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 32, 'weights': 'uniform'}
Best Precision Score: 0.5341751846788305


In [14]:
tuned_knn= gs_knn.best_estimator_

sfs= SequentialFeatureSelector(tuned_knn, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features = sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]
tuned_knn.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn, labels= tuned_knn.classes_)
print(classification_report(y_test, y_pred_tuned_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn))

C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is i

C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is i

              precision    recall  f1-score   support

           0       0.64      0.36      0.46      1125
           1       0.36      0.64      0.46       628

    accuracy                           0.46      1753
   macro avg       0.50      0.50      0.46      1753
weighted avg       0.54      0.46      0.46      1753

Precision: 0.35879218472468916
Accuracy: 0.46035367940673133
AUC: 0.49987756546355266


--- GaussianNB ---

In [15]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()

sfs= SequentialFeatureSelector(nb, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features= sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

nb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_nb, labels= nb.classes_)
print(classification_report(y_test, y_pred_nb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb))

              precision    recall  f1-score   support

           0       0.65      0.41      0.50      1125
           1       0.36      0.60      0.45       628

    accuracy                           0.48      1753
   macro avg       0.50      0.50      0.47      1753
weighted avg       0.54      0.48      0.48      1753

Precision: 0.36076555023923446
Accuracy: 0.4757558471192242
AUC: 0.49109271054493975


--- Random Forest ---

In [16]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)

sfs= SequentialFeatureSelector(rf, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features= sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

In [17]:
cm= confusion_matrix(y_test, y_pred_rf, labels= rf.classes_)
print(classification_report(y_test, y_pred_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf))

              precision    recall  f1-score   support

           0       0.62      0.25      0.36      1125
           1       0.35      0.73      0.47       628

    accuracy                           0.42      1753
   macro avg       0.49      0.49      0.42      1753
weighted avg       0.52      0.42      0.40      1753

Precision: 0.3510392609699769
Accuracy: 0.4209925841414718
AUC: 0.4806744515215853


In [18]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Precision Score: 0.6439829168190435


In [19]:
tuned_rf= gs_tuned_rf.best_estimator_

sfs= SequentialFeatureSelector(tuned_rf, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features = sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]
tuned_rf.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

              precision    recall  f1-score   support

           0       0.62      0.26      0.36      1125
           1       0.35      0.72      0.47       628

    accuracy                           0.42      1753
   macro avg       0.48      0.49      0.42      1753
weighted avg       0.52      0.42      0.40      1753

Precision: 0.3501552795031056
Accuracy: 0.42156303479749
AUC: 0.4797367303609342


--- AdaBoost ---

In [20]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)

sfs= SequentialFeatureSelector(ada, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features= sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

In [21]:
cm= confusion_matrix(y_test, y_pred_ada, labels= ada.classes_)
print(classification_report(y_test, y_pred_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada))

              precision    recall  f1-score   support

           0       0.66      0.24      0.35      1125
           1       0.36      0.78      0.50       628

    accuracy                           0.43      1753
   macro avg       0.51      0.51      0.42      1753
weighted avg       0.55      0.43      0.40      1753

Precision: 0.3641102010424423
Accuracy: 0.43354249857387334
AUC: 0.5179327671620666


In [22]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.6507501331474375


In [23]:
tuned_ada= gs_ada.best_estimator_

sfs= SequentialFeatureSelector(tuned_ada, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features = sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]
tuned_ada.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

              precision    recall  f1-score   support

           0       0.67      0.22      0.33      1125
           1       0.37      0.81      0.50       628

    accuracy                           0.43      1753
   macro avg       0.52      0.51      0.42      1753
weighted avg       0.56      0.43      0.39      1753

Precision: 0.36611629576453697
Accuracy: 0.4289788933257273
AUC: 0.5073333333333333


--- GradientBoosting ---

In [24]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)

sfs= SequentialFeatureSelector(grb, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features= sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.67      0.13      0.22      1125
           1       0.36      0.88      0.51       628

    accuracy                           0.40      1753
   macro avg       0.51      0.51      0.37      1753
weighted avg       0.56      0.40      0.33      1753

Precision: 0.3616742969260955
Accuracy: 0.4004563605248146
AUC: 0.4999922151450814


In [25]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.6688128041686718


In [ ]:
tuned_grb= gs_grb.best_estimator_

sfs= SequentialFeatureSelector(tuned_grb, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features = sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]
tuned_grb.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

--- XGB ---

In [ ]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)

sfs= SequentialFeatureSelector(xgb, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features= sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

In [ ]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_

sfs= SequentialFeatureSelector(tuned_xgb, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features = sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]
tuned_xgb.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

--- LGBM ---

In [ ]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)

sfs= SequentialFeatureSelector(lgm, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features= sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]


lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

In [ ]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_

sfs= SequentialFeatureSelector(tuned_lgm, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features = sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]
tuned_lgm.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

--- CatBoost ---

In [ ]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)

sfs= SequentialFeatureSelector(cat, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features= sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

In [ ]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_

sfs= SequentialFeatureSelector(tuned_cat, n_features_to_select= 10, direction= 'forward', scoring= 'precision')
sfs.fit(X_sm_tl, y_sm_tl)
selected_features = sfs.get_support()

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]
tuned_cat.fit(X_sm_tl_selected, y_sm_tl)


y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

--- Visualisation ---

In [ ]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

sfs_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
sfs_precision= sfs_precision.sort_values(by= 'Precision Score', ascending=False)
print(sfs_precision)

In [ ]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

sfs_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
sfs_accuracy= sfs_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(sfs_accuracy)

In [ ]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

sfs_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
sfs_auc= sfs_auc.sort_values(by= 'AUC Score', ascending=False)
print(sfs_auc)